No need to run again once downloaded:

In [5]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
import gensim
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
import pickle

In [13]:
#nltk.download()

In [7]:
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [8]:
#Get data
test1 = pickle.load(open('test1.p','rb'))
train1 = pickle.load(open('train1.p','rb'))
unlabeled_train = pickle.load(open('unlabeled_train.p', 'rb'))

### This function replaces all the Kaggle word parsing/cleaning by importing KaggleWord2VecUtility package :

Need to use this function instead of parsing by sentence:
Note* can choose to remove stopwords.

In [9]:
def getCleanReviews(reviews):
    clean_reviews = []
    for review in reviews["review"]:
        clean_reviews.append( KaggleWord2VecUtility.review_to_wordlist( review, remove_stopwords=False ))
    return clean_reviews

def getCleanReviewsNoStops(reviews):
    clean_reviews = []
    for review in reviews["review"]:
        clean_reviews.append( KaggleWord2VecUtility.review_to_wordlist( review, remove_stopwords=True ))
    return clean_reviews

# Word2Vec inversion

Here is some stuff to split the labelled training set into positive and negative reviews. NOTE: takes a long time to run.

Gets a list of positive review indexes and negative review indexes. Takes those respective reviews and parses them into words and appends to a list of positive and negative words, respectively. Also does the same for the unlabelled training reviews and their words.

In [10]:
# ****** Split the labeled and unlabeled training sets into clean words
#
inxs_pos = np.where(train1["sentiment"] == 1)[0].tolist()
inxs_neg = np.where(train1["sentiment"] == 0)[0].tolist()

print "Parsing words from training set"
reviews_pos = train1.iloc[inxs_pos]
words_pos = getCleanReviews(reviews_pos)

reviews_neg = train1.iloc[inxs_neg]
words_neg = getCleanReviews(reviews_neg)   
    
print "Parsing words from unlabeled set"
words_unlabelled = getCleanReviews(unlabeled_train)

Parsing words from training set


/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Parsing words from unlabeled set


Now do the same for all sentences without splitting to build a vocabulary:

In [12]:
# ****** Split the labeled and unlabeled training sets into clean words, don't separate by class
#
words = [] #initialize an empty list

print "Parsing words from training set"
words += getCleanReviews(train1)

print "Parsing words from unlabeled set"
words += getCleanReviews(unlabeled_train)

Parsing words from training set
Parsing words from unlabeled set


Some stuff from Taddy's code (much simpler than Kaggle method). 

In [14]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [15]:
basemodel.build_vocab(words) 

Train 2 models off of the base model (one pos & one neg). This changes now that we have words only.

In [16]:
from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(words_neg, total_examples=len(words_neg) )
models[1].train(words_pos, total_examples=len(words_pos) )

3359795

### Inversion of the distributed representations

At this point, we have 5 different word2vec language representations. Each 'model' has been trained conditional (i.e., limited to) text from a specific star rating. We will apply Bayes rule to go from p(text|stars) to p(stars|text).
For any new word we can obtain its likelihood (lhd; actually, the composite likelihood approximation; see the paper) using the score function in the word2vec class. We get the likelihood for each word in the first test review, then convert to a probability over star ratings. Every word in the review is evaluated separately and the final star rating of the review is an average vote of all the words. This is all in the following handy wrapper.

In [17]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of words
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.

***HERE is where I need to modify how the class probabilities are calculated. 
Must average across words now instead of sentences.
"""

import pandas as pd # for quick summing within doc

def docprobwords(docs,mods):
    sentlist = [s for d in docs for s in d]
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    lhd = np.exp(llhd - llhd.max(axis=0))
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

### Test set example
As an example, we apply the inversion on the full test set.

In [18]:
# read in the test set as a list of lists of words
docs = getCleanReviews(test1)

In [19]:
# get the probs (note we give docprob our test set plus the models)
probswords = docprobwords(docs[0:2], models)

In [20]:
probswords

,0,1
doc,,
0,0.537504,0.462496
1,0.425948,0.574052


In [21]:
# get the probs (note we give docprob our test set plus the models)
probswords = docprobwords(docs, models)

A numpy array of ones for the number of docs that we're predicting:

In [22]:
predswords = np.ones((probswords.shape[0]))

In [23]:
print np.shape(predswords)

(12500,)


In [24]:
predswords[np.where(probswords.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

In [25]:
predswords

array([ 0.,  1.,  1., ...,  1.,  1.,  1.])

In [26]:
print predswords.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [27]:
print np.size(np.where(predswords == test1["sentiment"]))*1./np.size(predswords)

0.48728


### Now testing without stopwords (*slight* improvement)

In [28]:
# ****** Split the labeled and unlabeled training sets into clean words

print "Parsing words from training set"
words_pos_nostops = getCleanReviewsNoStops(reviews_pos)

words_neg_nostops = getCleanReviewsNoStops(reviews_neg)   
    
print "Parsing words from unlabeled set"
words_unlabelled_nostops = getCleanReviewsNoStops(unlabeled_train)

Parsing words from training set
Parsing words from unlabeled set


In [29]:
# ****** Split the labeled and unlabeled training sets into clean words, don't separate by class
#
words_nostops = [] #initialize an empty list

print "Parsing words from training set"
words_nostops += getCleanReviewsNoStops(train1)

print "Parsing words from unlabeled set"
words_nostops += getCleanReviewsNoStops(unlabeled_train)

Parsing words from training set
Parsing words from unlabeled set


In [30]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [31]:
basemodel.build_vocab(words_nostops) 

In [32]:
from copy import deepcopy
models_nostops = [deepcopy(basemodel) for i in range(2)]
models_nostops[0].train(words_neg_nostops, total_examples=len(words_neg_nostops) )
models_nostops[1].train(words_pos_nostops, total_examples=len(words_pos_nostops) )

2188011

In [33]:
# read in the test set as a list of lists of words
docs_nostops = getCleanReviewsNoStops(test1)

In [34]:
# get the probs (note we give docprob our test set plus the models)
probswords = docprobwords(docs_nostops, models_nostops)

In [35]:
predswords = np.ones((probswords.shape[0]))

In [36]:
predswords[np.where(probswords.iloc[:,0] > 0.5)] = 0 # The second column is actually the negative model

In [37]:
print predswords.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [38]:
print np.size(np.where(predswords == test1["sentiment"]))*1./np.size(predswords)

0.50192


### The Word2Vec Paper/Kaggle way: stop words not removed from training, but from test set.

Performance is in the middle of the two. Hard to tell if this or all stop words removed is better. Need to test on other data.

In [39]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [40]:
basemodel.build_vocab(words) 

In [41]:
from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(words_neg, total_examples=len(words_neg) )
models[1].train(words_pos, total_examples=len(words_pos) )

3359617

In [42]:
# get the probs (note we give docprob our test set plus the models)
probswords = docprobwords(docs_nostops, models)

In [43]:
predswords = np.ones((probswords.shape[0]))

In [44]:
predswords[np.where(probswords.iloc[:,0] > 0.5)] = 0 # The second column is actually the negative model

In [45]:
print predswords.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [46]:
print np.size(np.where(predswords == test1["sentiment"]))*1./np.size(predswords)

0.486
